# train

In [1]:
import yaml
import os

with open("./config.yaml", 'rb') as f:
    config = yaml.load(f)

In [2]:
OUTPUT_ROOT = config['IO_OPTION']['OUTPUT_ROOT']
os.makedirs(OUTPUT_ROOT, exist_ok=True)

## load library

In [3]:
# python default library
import os
import shutil
import datetime
import sys
import pickle

# general analysis tool-kit
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# pytorch
import torch
from torch import optim, nn
from torch.utils.tensorboard import SummaryWriter

from torchsummary import summary

# etc
import yaml
yaml.warnings({'YAMLLoadWarning': False})
import mlflow
from collections import defaultdict
from scipy.stats import zscore

# original library

import common as com
import pytorch_modeler as modeler
from pytorch_model import DAGMM as Model
from pytorch_utils import filtered_load_model
import models

import librosa
import IPython
import librosa.display

## load config and set logger

In [4]:
#with open("./config.yaml", 'rb') as f:
#    config = yaml.load(f)

log_folder = config['IO_OPTION']['OUTPUT_ROOT']+'/{0}.log'.format(datetime.date.today())
logger = com.setup_logger(log_folder, '00_train.py')

/home/hiroki/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Setting

In [5]:
# Setting seed
modeler.set_seed(42)

In [6]:
############################################################################
# Setting I/O path
############################################################################
# input dirs
INPUT_ROOT = config['IO_OPTION']['INPUT_ROOT']
dev_path = INPUT_ROOT + "/dev_data"
add_dev_path = INPUT_ROOT + "/add_dev_data"
# machine type
MACHINE_TYPE = config['IO_OPTION']['MACHINE_TYPE']
machine_types = os.listdir(dev_path)
# output dirs
OUTPUT_ROOT = config['IO_OPTION']['OUTPUT_ROOT']
MODEL_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/models'
TB_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/tb'
PKL_DIR = OUTPUT_ROOT + '/pkl'
#os.makedirs(OUTPUT_ROOT, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(TB_DIR, exist_ok=True)
os.makedirs(PKL_DIR, exist_ok=True)
# copy config
shutil.copy('./config.yaml', OUTPUT_ROOT)

'/media/hiroki/working/research/dcase2020/result/2D/DAGMM/add_metaAE/v1/config.yaml'

## make path list and train/valid split

In [7]:
############################################################################
# make path set and train/valid split
############################################################################
'''
train_paths[machine_type]['train' or 'valid'] = path
'''
dev_train_paths = {}
add_train_paths = {}
train_paths = {}

In [8]:
for machine_type in machine_types:
    # dev train
    dev_train_paths = ["{}/{}/train/".format(dev_path, machine_type) + file for file in os.listdir("{}/{}/train".format(dev_path, machine_type))]
    dev_train_paths = sorted(dev_train_paths)
    # add_dev train
    add_train_paths = ["{}/{}/train/".format(add_dev_path, machine_type) + file for file in os.listdir("{}/{}/train".format(add_dev_path, machine_type))]
    add_train_paths = sorted(add_train_paths)
    # valid
    dev_valid_paths = ["{}/{}/test/".format(dev_path, machine_type) + file for file in os.listdir("{}/{}/test".format(dev_path, machine_type))]
    dev_valid_paths = sorted(dev_valid_paths)
    
    train_paths[machine_type] = {}
    train_paths[machine_type]['train'] = dev_train_paths + add_train_paths
    train_paths[machine_type]['valid'] = dev_valid_paths

## training

In [9]:
#############################################################################
# run
#############################################################################
def run(machine_type):
    com.tic()
    logger.info('TARGET MACHINE_TYPE: {0}'.format(machine_type))
    logger.info('MAKE DATA_LOADER')
    # dev_train_paths
    dataloaders_dict = modeler.make_dataloader(train_paths, machine_type)
    # define writer for tensorbord
    os.makedirs(TB_DIR+'/'+machine_type, exist_ok=True)         # debug
    tb_log_dir = TB_DIR + '/' + machine_type
    writer = SummaryWriter(log_dir = tb_log_dir)
    logger.info('TRAINING')
    # parameter setting
    net = Model(sample_rate=config['preprocessing']['sample_rate'],
                window_size=config['preprocessing']['window_size'],
                hop_size=config['preprocessing']['hop_size'],
                mel_bins=config['preprocessing']['mel_bins'],
                fmin=config['preprocessing']['fmin'],
                fmax=config['preprocessing']['fmax'],
                latent_size=config['fit']['latent_size'],
                mixture_size=config['fit']['mixture_size'])
    #pretrained_dict = torch.load(config['IO_OPTION']['PREMODEL_PATH'])
    #net = filtered_load_model(net, pretrained_dict)
    optimizer = optim.Adam(net.parameters(), lr=1e-3)
    num_epochs = config['fit']['num_epochs']
    scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e3, 
                                              max_lr=1e-3, epochs=num_epochs, steps_per_epoch=len(dataloaders_dict['train']))
    criterion = nn.MSELoss()
    history = modeler.train_net(net, dataloaders_dict, criterion, optimizer, scheduler, num_epochs, writer)
    # output
    model = history['model']
    model_out_path = MODEL_DIR+'/{}_model.pth'.format(machine_type)
    torch.save(model.state_dict(), model_out_path)
    logger.info('\n success:{0} \n'.format(machine_type) + \
                    'model_out_path ==> \n {0}'.format(model_out_path))
    #  close writer for tensorbord
    writer.close()
    #modeler.mlflow_log(history, config, machine_type, model_out_path, tb_log_dir)
    com.toc()
    return history

In [10]:
machine_types

['fan', 'pump', 'slider', 'ToyCar', 'ToyConveyor', 'valve']

In [11]:
for machine_type in machine_types:
    history = run(machine_type)
    #with open('{}/{}_history.pkl'.format(PKL_DIR, machine_type), 'wb') as file:
    #    pickle.dump(history , file)

2020-11-28 16:35:39,958 - 00_train.py - INFO - TARGET MACHINE_TYPE: fan
2020-11-28 16:35:39,959 - 00_train.py - INFO - MAKE DATA_LOADER
2020-11-28 16:35:40,968 - 00_train.py - INFO - TRAINING


use: cuda:0


100%|██████████| 29/29 [00:02<00:00, 10.75it/s]
2020-11-28 16:36:12,111 - pytorch_modeler.py - INFO - Epoch 1/300:train_loss:20922.982246, tr_rec:37.223140, tr_eng:643.746529, tr_covd:4105706.066629, val_AUC:0.477767, val_pAUC:0.505571
100%|██████████| 29/29 [00:02<00:00, 11.65it/s]
2020-11-28 16:36:38,428 - pytorch_modeler.py - INFO - Epoch 2/300:train_loss:246.201453, tr_rec:27.067654, tr_eng:766.038214, tr_covd:2887.773151, val_AUC:0.504089, val_pAUC:0.500178
100%|██████████| 29/29 [00:02<00:00, 11.46it/s]
2020-11-28 16:37:04,752 - pytorch_modeler.py - INFO - Epoch 3/300:train_loss:110.236568, tr_rec:6.115046, tr_eng:691.259548, tr_covd:3112.033735, val_AUC:0.500114, val_pAUC:0.497693
100%|██████████| 29/29 [00:02<00:00, 11.66it/s]
2020-11-28 16:37:31,479 - pytorch_modeler.py - INFO - Epoch 4/300:train_loss:77.291309, tr_rec:2.235217, tr_eng:554.609078, tr_covd:2949.523285, val_AUC:0.486935, val_pAUC:0.508047
100%|██████████| 29/29 [00:02<00:00, 11.49it/s]
2020-11-28 16:37:58,299 - 

100%|██████████| 29/29 [00:04<00:00,  7.15it/s]
2020-11-28 16:50:16,613 - pytorch_modeler.py - INFO - Epoch 30/300:train_loss:160.118274, tr_rec:0.112097, tr_eng:-450.904844, tr_covd:40829.574592, val_AUC:0.556777, val_pAUC:0.510457
100%|██████████| 29/29 [00:04<00:00,  7.02it/s]
2020-11-28 16:50:54,781 - pytorch_modeler.py - INFO - Epoch 31/300:train_loss:-14.224563, tr_rec:0.110825, tr_eng:-341.266250, tr_covd:3456.872932, val_AUC:0.516099, val_pAUC:0.507128
100%|██████████| 29/29 [00:04<00:00,  7.09it/s]
2020-11-28 16:51:32,301 - pytorch_modeler.py - INFO - Epoch 32/300:train_loss:212.809115, tr_rec:0.109190, tr_eng:-459.446113, tr_covd:51200.272486, val_AUC:0.525500, val_pAUC:0.504577
100%|██████████| 29/29 [00:03<00:00,  7.33it/s]
2020-11-28 16:52:11,028 - pytorch_modeler.py - INFO - Epoch 33/300:train_loss:-46.326242, tr_rec:0.104555, tr_eng:-565.437413, tr_covd:1723.019273, val_AUC:0.524677, val_pAUC:0.510119
100%|██████████| 29/29 [00:04<00:00,  7.01it/s]
2020-11-28 16:52:49,32

100%|██████████| 29/29 [00:02<00:00, 10.39it/s]
2020-11-28 17:08:18,787 - pytorch_modeler.py - INFO - Epoch 59/300:train_loss:-53.865326, tr_rec:0.154656, tr_eng:-791.077327, tr_covd:4872.149145, val_AUC:0.500000, val_pAUC:0.500000
100%|██████████| 29/29 [00:02<00:00, 10.43it/s]
2020-11-28 17:08:45,788 - pytorch_modeler.py - INFO - Epoch 60/300:train_loss:110.645098, tr_rec:0.154642, tr_eng:-894.374042, tr_covd:39858.285722, val_AUC:0.500000, val_pAUC:0.500000
100%|██████████| 29/29 [00:02<00:00, 10.83it/s]
2020-11-28 17:09:13,059 - pytorch_modeler.py - INFO - Epoch 61/300:train_loss:-65.046035, tr_rec:0.154640, tr_eng:-756.854685, tr_covd:1995.957514, val_AUC:0.500000, val_pAUC:0.500000
100%|██████████| 29/29 [00:02<00:00, 10.59it/s]
2020-11-28 17:09:40,158 - pytorch_modeler.py - INFO - Epoch 62/300:train_loss:-68.797331, tr_rec:0.154599, tr_eng:-863.726314, tr_covd:3372.627047, val_AUC:0.500000, val_pAUC:0.500000
100%|██████████| 29/29 [00:02<00:00, 11.02it/s]
2020-11-28 17:10:07,050

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

history = run('ToyCar')
with open('{}/{}_history.pkl'.format(PKL_DIR, machine_type), 'wb') as file:
    pickle.dump(history , file)